En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "data/farmers-protest-tweets-2021-2-4.json"

En caso de que el archivo sea muchísimo más grande, este tipo de funciones iterativas deja de ser tan óptimo, por lo que podría pensarse en el uso de (Py)Spark. 

Usualmente en algún job de dataproc con un cluster efímero que permita el procesamiento distribuído de la data y posible carga a Bigquery por ejemplo.

O, si tenemos información que llega en Streaming, podría pensarse en el uso de Dataflow con Apache Beam.

Importación de las librerías

In [33]:
import json
import ujson
from collections import defaultdict, Counter
from datetime import datetime
from typing import List, Tuple, Optional
from pydantic import BaseModel, ValidationError, field_validator
import emoji
import regex


%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## Validador de esquema

Antes de iniciar con el desarrollo de las lógicas asociadas a la solución de los retos, considero fundamental validar el esquema del Json que se espera procesar. En este caso se usa Pydantic para valdiar los types y la existencia de los objetos y sub objetos.

In [3]:
class DescriptionUrlModel(BaseModel):
    """
    Modelo que representa una URL de descripción dentro de un tweet.

    Attributes:
        text (str): El texto de la URL.
        indices (List[int]): Lista de índices que indican la posición de la URL en el texto del tweet.
    """

    text: str
    indices: List[int]

class UserModel(BaseModel):
    """
    Modelo que representa un usuario de Twitter.

    Attributes:
        username (str): El nombre de usuario de Twitter.
        displayname (Optional[str]): El nombre de pantalla del usuario.
        id (int): El ID del usuario.
        description (Optional[str]): La descripción del usuario.
        rawDescription (Optional[str]): La descripción cruda del usuario.
        descriptionUrls (Optional[List[DescriptionUrlModel]]): Lista de URLs en la descripción del usuario.
        verified (Optional[bool]): Indicador de si el usuario está verificado.
        created (Optional[str]): La fecha de creación de la cuenta del usuario.
        followersCount (Optional[int]): La cantidad de seguidores del usuario.
        friendsCount (Optional[int]): La cantidad de amigos del usuario.
        statusesCount (Optional[int]): La cantidad de estados del usuario.
        favouritesCount (Optional[int]): La cantidad de favoritos del usuario.
        listedCount (Optional[int]): La cantidad de listas en las que aparece el usuario.
        mediaCount (Optional[int]): La cantidad de medios publicados por el usuario.
        location (Optional[str]): La ubicación del usuario.
        protected (Optional[bool]): Indicador de si la cuenta del usuario está protegida.
        linkUrl (Optional[str]): URL del perfil del usuario.
        linkTcourl (Optional[str]): URL acortada del perfil del usuario.
        profileImageUrl (Optional[str]): URL de la imagen del perfil del usuario.
        profileBannerUrl (Optional[str]): URL del banner del perfil del usuario.
        url (Optional[str]): URL del usuario.
    """

    username: str
    displayname: Optional[str] = None
    id: int
    description: Optional[str] = None
    rawDescription: Optional[str] = None
    descriptionUrls: Optional[List[DescriptionUrlModel]] = None
    verified: Optional[bool] = None
    created: Optional[str] = None
    followersCount: Optional[int] = None
    friendsCount: Optional[int] = None
    statusesCount: Optional[int] = None
    favouritesCount: Optional[int] = None
    listedCount: Optional[int] = None
    mediaCount: Optional[int] = None
    location: Optional[str] = None
    protected: Optional[bool] = None
    linkUrl: Optional[str] = None
    linkTcourl: Optional[str] = None
    profileImageUrl: Optional[str] = None
    profileBannerUrl: Optional[str] = None
    url: Optional[str] = None

class TweetModel(BaseModel):
    """
    Modelo que representa un tweet.

    Attributes:
        url (str): URL del tweet.
        date (str): Fecha y hora del tweet en formato ISO 8601.
        content (Optional[str]): Contenido del tweet.
        renderedContent (Optional[str]): Contenido renderizado del tweet.
        id (int): ID del tweet.
        user (UserModel): Información del usuario que publicó el tweet.
        outlinks (Optional[List[str]]): Lista de URLs externas incluidas en el tweet.
        tcooutlinks (Optional[List[str]]): Lista de URLs acortadas incluidas en el tweet.
        replyCount (Optional[int]): Cantidad de respuestas al tweet.
        retweetCount (Optional[int]): Cantidad de retweets del tweet.
        likeCount (Optional[int]): Cantidad de "me gusta" del tweet.
        quoteCount (Optional[int]): Cantidad de citas del tweet.
        conversationId (Optional[int]): ID de la conversación a la que pertenece el tweet.
        lang (Optional[str]): Idioma del tweet.
        source (Optional[str]): Fuente desde donde se publicó el tweet.
        sourceUrl (Optional[str]): URL de la fuente desde donde se publicó el tweet.
        sourceLabel (Optional[str]): Etiqueta de la fuente desde donde se publicó el tweet.
        media (Optional[List[dict]]): Lista de medios adjuntos al tweet.
        retweetedTweet (Optional[dict]): Información del tweet retuiteado, si aplica.
        quotedTweet (Optional[dict]): Información del tweet citado, si aplica.
        mentionedUsers (Optional[List[UserModel]]): Lista de usuarios mencionados en el tweet.
    """

    url: str
    date: str
    content: Optional[str] = None
    renderedContent: Optional[str] = None
    id: int
    user: UserModel
    outlinks: Optional[List[str]] = None
    tcooutlinks: Optional[List[str]] = None
    replyCount: Optional[int] = None
    retweetCount: Optional[int] = None
    likeCount: Optional[int] = None
    quoteCount: Optional[int] = None
    conversationId: Optional[int] = None
    lang: Optional[str] = None
    source: Optional[str] = None
    sourceUrl: Optional[str] = None
    sourceLabel: Optional[str] = None
    media: Optional[List[dict]] = None
    retweetedTweet: Optional[dict] = None
    quotedTweet: Optional[dict] = None
    mentionedUsers: Optional[List[UserModel]] = None

    @field_validator('date')
    def validate_date_format(cls, value):
        """
        Valida que la fecha esté en el formato ISO 8601 correcto.

        Args:
            value (str): La fecha en formato de cadena.

        Returns:
            str: La fecha validada.

        Raises:
            ValueError: Si la fecha no está en el formato correcto.
        """

        try:
            datetime.strptime(value, "%Y-%m-%dT%H:%M:%S%z")
        except ValueError:
            raise ValueError("Incorrect date format")
        return value

def validate_tweet(tweet: dict) -> bool:
    """
    Valida que un diccionario de tweet cumpla con el esquema definido por TweetModel.

    Args:
        tweet (dict): Diccionario que representa el tweet.

    Returns:
        bool: True si el tweet es válido, False si no lo es.

    Raises:
        ValidationError: Si el tweet no cumple con el esquema.
    """
    
    try:
        TweetModel(**tweet)
        return True
    except ValidationError as e:
        print(f"Validation error: {e}")
        return False

# 1. Top 10 fechas donde hay más tweets

## q1_time

### Desarrollo de la función

La función q1_time está optimizada para ser rápida en términos de tiempo de ejecución. Utiliza el módulo ujson (UltraJSON) para la lectura y decodificación de los datos JSON, lo cual es significativamente más rápido que el módulo json estándar de Python.

Cómo logra la optimización en tiempo
* **Lectura Rápida de Datos:**
Utiliza ujson.loads en lugar de json.loads, lo que reduce el tiempo de decodificación de los datos JSON.
* **Procesamiento Eficiente de Datos:**
Procesa los datos de manera eficiente con un único bucle para contar los tweets por usuario y fecha.
* **Uso de Estructuras de Datos Adecuadas:**
Utiliza defaultdict y Counter para gestionar y contar los tweets de manera eficiente.

In [ ]:
# Optimización por tiempo
def q1_time(file_path: str) -> List[Tuple[datetime, str]]:
    """
    Procesa un archivo JSON línea por línea, analiza los tweets contenidos en él
    y devuelve una lista con las 10 fechas con más tweets y el usuario que más tweets
    publicó en cada una de esas fechas.

    Args:
        file_path (str): La ruta al archivo JSON que contiene los tweets. Cada línea del archivo
                         debe ser un objeto JSON que representa un tweet.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha
                                         y el nombre de usuario del usuario que más tweets publicó
                                         en esa fecha. La lista está ordenada por la cantidad de tweets
                                         en orden descendente, incluyendo solo las 10 fechas principales.
    """
    
    # Se usa ujson para la lectura del archivo
    with open(file_path, 'r') as file:
        data = [ujson.loads(line) for line in file]
    
    # Contador para rastrear la cantidad de tweets por usuario en cada fecha
    date_user_count = defaultdict(lambda: Counter())
    
    for tweet in data:
        # Validación del schema
        if not validate_tweet(tweet):
            continue

        # Extraer la fecha del tweet y convertirla a un objeto de fecha
        date = datetime.strptime(tweet["date"], "%Y-%m-%dT%H:%M:%S%z").date()

        # Extraer el nombre de usuario del tweet
        user = tweet["user"]["username"]

        # Incrementar el contador para el usuario en la fecha correspondiente
        date_user_count[date][user] += 1
    
    # Ordenar por cantidad de tweets y obtener las 10 fechas principales
    top_dates = sorted(date_user_count.items(), key=lambda x: sum(x[1].values()), reverse=True)[:10]
    
    # Obtener el usuario con más publicaciones en cada fecha
    result = [(date, user_count.most_common(1)[0][0]) for date, user_count in top_dates]
    return result


### Test y evaluación

Revisión de los resultados

In [ ]:
q1_time(file_path)

Medición de tiempo promedio con su derivación estandard para 7 ejecuciones

In [ ]:
%timeit q1_time(file_path)

Medición de la memoria usada

In [ ]:
%memit q1_time(file_path)

## q1_memory

### Desarrollo de la función

La función q1_memory está optimizada para utilizar menos memoria en comparación con una implementación estándar. Aunque no usa técnicas avanzadas de reducción de memoria, se asegura de que la memoria utilizada sea eficiente y no excesiva.

Cómo logra la optimización en memoria
* **Lectura y Decodificación Controlada:**
Lee y decodifica el archivo JSON línea por línea para evitar cargar el archivo completo en memoria a la vez.
* **Gestión Eficiente de Contadores:**
Utiliza defaultdict y Counter para rastrear el número de tweets por usuario y fecha, lo que ayuda a mantener un uso de memoria controlado.
* **Optimización de la Estructura de Datos:**
La estructura de los contadores asegura que solo se almacene la información necesaria para el análisis, minimizando el uso de memoria.

In [ ]:
# Optimización por memoria
def q1_memory(file_path: str) -> List[Tuple[datetime, str]]:
    """
    Procesa un archivo JSON línea por línea, analiza los tweets contenidos en él
    y devuelve una lista con las 10 fechas con más tweets y el usuario que más tweets
    publicó en cada una de esas fechas. Esta implementación está optimizada para el
    uso eficiente de memoria.

    Args:
        file_path (str): La ruta al archivo JSON que contiene los tweets. Cada línea del archivo
                         debe ser un objeto JSON que representa un tweet.

    Returns:
        List[Tuple[datetime.date, str]]: Una lista de tuplas, donde cada tupla contiene una fecha
                                         y el nombre de usuario del usuario que más tweets publicó
                                         en esa fecha. La lista está ordenada por la cantidad de tweets
                                         en orden descendente, incluyendo solo las 10 fechas principales.
    """

    # Contador para rastrear la cantidad de tweets por usuario en cada fecha
    date_user_count = defaultdict(Counter)

    # Leer el archivo línea por línea y procesar cada tweet
    with open(file_path, 'r') as file:
        for line in file:
            tweet = json.loads(line)  # Usar ujson para la carga rápida de JSON
            if not validate_tweet(tweet):  # Validar el esquema del tweet
                continue
            date = datetime.strptime(tweet["date"], "%Y-%m-%dT%H:%M:%S%z").date()  # Convertir la fecha a objeto de fecha
            user = tweet["user"]["username"]  # Obtener el nombre de usuario
            date_user_count[date][user] += 1  # Incrementar el contador para el usuario en la fecha correspondiente

    # Ordenar por cantidad de tweets y obtener las 10 fechas principales
    top_dates = sorted(date_user_count.items(), key=lambda x: sum(x[1].values()), reverse=True)[:10]
    
    # Obtener el usuario con más publicaciones en cada fecha
    result = [(date, user_count.most_common(1)[0][0]) for date, user_count in top_dates]
    return result

### Test y evaluación

Revisión de los resultados

In [ ]:
q1_memory(file_path)

Medición de tiempo promedio con su derivación estandard para 7 ejecuciones

In [ ]:
%timeit q1_memory(file_path)

Medición de la memoria usada

In [ ]:
%memit q1_memory(file_path)

## Comparación de resultados

| Función     | Tiempo de Ejecución             | Memoria Máxima (peak) | 
|-------------|---------------------------------|-----------------------|
| `q1_time`   | 9.72 s ± 114 ms per loop        | 1338.06 MiB           |
| `q1_memory` | 11.9 s ± 151 ms per loop        | 142.88 MiB            |


# 2. Top 10 emojis más usados

## q2_time

### Desarrollo de la función

La función q2_time procesa un archivo JSON que contiene tweets para contar los emojis más usados de una manera eficiente en términos de tiempo. Los pasos son los siguientes:

* **Lectura del Archivo:**

Se abre el archivo especificado por file_path.
Se lee todo el contenido del archivo y se convierte cada línea en un objeto JSON usando ujson.loads(line).

* **Procesamiento de los Tweets:**

Se itera sobre cada tweet.
Se valida el esquema del tweet usando una función validate_tweet. Si el tweet no es válido, se salta.
Se obtiene el contenido del tweet y se extraen los emojis usando la función extract_emojis.
Se actualiza un contador (emoji_count) con los emojis encontrados.

* **Obtención de los Emojis más Comunes:**

Se obtienen los 10 emojis más comunes usando emoji_count.most_common(10).
La función devuelve esta lista de los emojis más usados junto con sus conteos.

In [4]:
emoji_pattern = regex.compile(r'\X')

def extract_emojis(text):
    """
    Extrae todos los emojis de un texto dado utilizando una expresión regular,
    excluyendo los modificadores de tono de piel.

    Args:
        text (str): El texto del cual extraer emojis.

    Returns:
        List[str]: Una lista de emojis encontrados en el texto, sin los modificadores de tono de piel.
    """
    emojis = [word for word in emoji_pattern.findall(text) if any(char in emoji.EMOJI_DATA for char in word)]
    return emojis

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo JSON línea por línea, analiza el contenido de los tweets
    y devuelve una lista con los 10 emojis más usados y su respectivo conteo.

    Args:
        file_path (str): La ruta al archivo JSON que contiene los tweets. Cada línea del archivo
                         debe ser un objeto JSON que representa un tweet.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas, donde cada tupla contiene un emoji y su conteo.
                               La lista está ordenada por conteo en orden descendente, incluyendo solo los 10 emojis principales.
    """
    
    # Leer y procesar el archivo en una sola pasada para mejorar la eficiencia
    with open(file_path, 'r') as file:
        data = [ujson.loads(line) for line in file]

    emoji_count = Counter()

    for tweet in data:
        if not validate_tweet(tweet):
            continue

        content = tweet.get("content", "")
        emojis = extract_emojis(content)
        emoji_count.update(emojis)
        
    result = emoji_count.most_common(10)
    return result

### Test y evaluación

Revisión de los resultados

In [5]:
q2_time(file_path)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🤣', 1668),
 ('✊', 1642),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040),
 ('👇', 873)]

Medición de tiempo promedio con su derivación estandard para 7 ejecuciones

In [6]:
%timeit q2_time(file_path)

24.3 s ± 238 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Medición de la memoria usada

In [7]:
%memit q2_time(file_path)

peak memory: 1346.95 MiB, increment: 1228.37 MiB


## q2_memory

### Desarrollo de la función

La función q2_memory también procesa un archivo JSON que contiene tweets para contar los emojis más usados, pero está optimizada para un uso eficiente de la memoria. Los pasos son los siguientes:

* **Lectura del Archivo:**

Se abre el archivo especificado por file_path.
En lugar de cargar todo el archivo en memoria de una vez, se lee línea por línea.

* **Procesamiento de los Tweets:**

Para cada línea del archivo, se convierte la línea en un objeto JSON usando json.loads(line).
Se valida el esquema del tweet usando una función validate_tweet. Si el tweet no es válido, se salta.
Se obtiene el contenido del tweet y se extraen los emojis directamente dentro de la función usando una versión interna de extract_emojis.
Se actualiza un contador (emoji_count) con los emojis encontrados.

* **Obtención de los Emojis más Comunes:**

Se obtienen los 10 emojis más comunes usando emoji_count.most_common(10).
La función devuelve esta lista de los emojis más usados junto con sus conteos.

In [8]:
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo JSON línea por línea, analiza el contenido de los tweets
    y devuelve una lista con los 10 emojis más usados y su respectivo conteo.
    Esta implementación está optimizada para el uso eficiente de memoria.

    Args:
        file_path (str): La ruta al archivo JSON que contiene los tweets. Cada línea del archivo
                         debe ser un objeto JSON que representa un tweet.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas, donde cada tupla contiene un emoji y su conteo.
                               La lista está ordenada por conteo en orden descendente, incluyendo solo los 10 emojis principales.
    """
    emoji_count = Counter()

    with open(file_path, 'r') as file:
        for line in file:
            tweet = json.loads(line)
            if not validate_tweet(tweet):  # Validar el esquema del tweet
                continue
            content = tweet.get("content", "")
            
            emoji_list = []
            data = regex.findall(r'\X', content)
            for word in data:
                if any(char in emoji.EMOJI_DATA  for char in word):
                    emoji_list.append(word)

            emoji_count.update(emoji_list)

    top_emojis = emoji_count.most_common(10)
    return top_emojis

### Test y evaluación

Revisión de los resultados

In [9]:
q2_memory(file_path)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🤣', 1668),
 ('✊', 1642),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040),
 ('👇', 873)]

Medición de tiempo promedio con su derivación estandard para 7 ejecuciones

In [10]:
%timeit q2_memory(file_path)

26.6 s ± 167 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Medición de la memoria usada

In [11]:
%memit q2_memory(file_path)

peak memory: 120.07 MiB, increment: 0.01 MiB


## Comparación de resultados

| Función     | Tiempo de Ejecución             | Memoria Máxima (peak) | 
|-------------|---------------------------------|-----------------------|
| `q2_time`   | 24.3 s ± 238 ms per loop        | 1346.95 MiB           |
| `q2_memory` | 26.6 s ± 167 ms per loop        | 120.07 MiB            |


# 3. Top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones

## q3_time

### Desarrollo de la función

In [29]:
# Precompilar la expresión regular para encontrar menciones
mention_pattern = regex.compile(r'@\w+')

def extract_mentions(text: str) -> List[str]:
    """
    Extrae todas las menciones (@) de un texto dado utilizando una expresión regular.
    
    Args:
        text (str): El texto del cual extraer menciones.

    Returns:
        List[str]: Una lista de menciones encontradas en el texto.
    """
    mentions = mention_pattern.findall(text)
    return [mention[1:] for mention in mentions] 


def q3_time(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo JSON línea por línea, analiza el contenido de los tweets
    y devuelve una lista con los 10 usuarios más mencionados y su respectivo conteo.

    Args:
        file_path (str): La ruta al archivo JSON que contiene los tweets. Cada línea del archivo
                         debe ser un objeto JSON que representa un tweet.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas, donde cada tupla contiene un usuario y su conteo.
                               La lista está ordenada por conteo en orden descendente, incluyendo solo los 10 usuarios principales.
    """
    
    # Leer y procesar el archivo en una sola pasada para mejorar la eficiencia
    with open(file_path, 'r') as file:
        data = [ujson.loads(line) for line in file]
    
    mention_count = Counter()    

    for tweet in data:
                
        if not validate_tweet(tweet):
            continue
        content = tweet.get("content", "")
        mentions = extract_mentions(content)
        mention_count.update(mentions)

    result = mention_count.most_common(10)
    return result

### Test y evaluación

Revisión de los resultados

In [30]:
q3_time(file_path)

[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

Medición de tiempo promedio con su derivación estandard para 7 ejecuciones

In [24]:
%timeit q3_time(file_path)

8.49 s ± 67.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Medición de la memoria usada

In [27]:
%memit q3_time(file_path)

peak memory: 1348.21 MiB, increment: 1215.17 MiB


## q3_memory

### Desarrollo de la función

In [31]:
# Precompilar la expresión regular para encontrar menciones
mention_pattern = regex.compile(r'@\w+')

def extract_mentions(text: str) -> List[str]:
    """
    Extrae todas las menciones (@) de un texto dado utilizando una expresión regular.
    
    Args:
        text (str): El texto del cual extraer menciones.

    Returns:
        List[str]: Una lista de menciones encontradas en el texto.
    """

    mentions = mention_pattern.findall(text)
    return [mention[1:] for mention in mentions] 

def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    """
    Procesa un archivo JSON línea por línea, analiza el contenido de los tweets
    y devuelve una lista con los 10 usuarios más mencionados y su respectivo conteo.
    Esta implementación está optimizada para el uso eficiente de memoria.

    Args:
        file_path (str): La ruta al archivo JSON que contiene los tweets. Cada línea del archivo
                         debe ser un objeto JSON que representa un tweet.

    Returns:
        List[Tuple[str, int]]: Una lista de tuplas, donde cada tupla contiene un usuario y su conteo.
                               La lista está ordenada por conteo en orden descendente, incluyendo solo los 10 usuarios principales.
    """
    
    mention_count = Counter()

    with open(file_path, 'r') as file:
        for line in file:
            tweet = json.loads(line)
            if not validate_tweet(tweet):
                continue
            content = tweet.get("content", "")
            mentions = extract_mentions(content)
            mention_count.update(mentions)

    return mention_count.most_common(10)

## Test y evaluación

Revisión de los resutlados

In [32]:
q3_memory(file_path)

[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]

Medición de tiempo promedio con su derivación estandard para 7 ejecuciones

In [20]:
%timeit q3_memory(file_path)

10.2 s ± 63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Medición de la memoria usada

In [21]:
%memit q3_memory(file_path)

peak memory: 118.32 MiB, increment: 0.00 MiB


## Comparación de resultados

| Función     | Tiempo de Ejecución             | Memoria Máxima (peak) | 
|-------------|---------------------------------|-----------------------|
| `q3_time`   | 8.49 s ± 67.7 ms per loop       | 1348.21 MiB           |
| `q3_memory` | 10.2 s ± 63 ms per loop         | 118.32 MiB            |


# Conclusiones

Debido al tamaño del archivo y limitación de recursos, es complejo optimizar el tiempo de ejecución, pero la memoria sí pudo ser optimizada de forma satisfactoria.

Se trató de usar librerías simples y que no dependan de otras con el fin de poder usar el código para crear las Cloud Functions sin mayor inconveniente.

# Post Request

In [1]:
import requests

url = "https://advana-challenge-check-api-cr-k4hdbggvoq-uc.a.run.app/data-engineer"
payload = {
    "name": "Juan Diego Gallego",
    "mail": "jgallegovillada@gmail.com",
    "github_url": "https://github.com/juand-gv/GCPChallengeDE.git"
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    print("POST request was successful!")
    print("Response:", response.text)
else:
    print("POST request failed with status code:", response.status_code)

POST request was successful!
Response: {"status":"OK","detail":"your request was received"}
